In [1]:
# Import packages
import os
import time
from openai import AzureOpenAI
import numpy as np
from dotenv import load_dotenv

In [2]:
# Load environment variables from the .env file
load_dotenv()

True

In [3]:
api_key = os.getenv('AZUREOPENAI_API_KEY')
api_version = os.getenv('AZUREOPENAI_API_VERSION')
azure_endpoint = os.getenv('AZUREOPENAI_API_ENDPOINT')

In [4]:
# Create Azure OpenAI client
# Make sure the environment variables are created
client = AzureOpenAI(
    api_key=api_key,
    api_version=api_version,
    azure_endpoint = azure_endpoint
    )

In [5]:
# Define chat completion function
def completeChat(prompt, style, client, model="gpt-4o-mini"):
    # Execute API call
    result = client.chat.completions.create(
        model=model,
        messages= [
            {
                "role": "system",
                "content": style,
            },
            {
                "role": "user",
                "content": prompt,
            },
        ],
        max_tokens=1000,
        temperature=0,
        top_p=1,
        frequency_penalty=0,
        presence_penalty=0,
        stop=None,
        stream=False,
        seed=42,
        n=1,
    )

    # Extract the response
    response = result.choices[0].message.content.strip()

    return response

In [6]:
# Define text embedding function
def embedText(text, client, model="text-embedding-ada-002"):
    # Execute API call
    result = client.embeddings.create(
        model=model,
        input=text
    )

    # Extract and normalize the embeddings
    embedding = np.array(result.data[0].embedding)
    embedding /= np.linalg.norm(embedding)

    return embedding

In [7]:
# Create prompt

example = """

{
	"Product Info" : {
		"Product Sheet" : {
			"Brand" : "Elola Beauté",
			"Product name" : "Shampoing Bouclés",
			"Marketing Description" : {
				"EN" : "Silk shampoo that regenerates and gently cleanses curly and curly hair"
			},
			"Key ingredients" : {
				"EN" : [
					"Silk"
				]
			},
			"Price (euros)" : "28,99",
			"Quantity (ml)" : "500 ml",
			"Category" : {
				"EN" : "Shampoo"
			},
			"Ages involved" : {
				"EN" : [
					"13-17 years",
					"18-24 years",
					"25-44 years",
					"45-64 years",
					"65 years and over"
				]
			},
			"Suitable for pregnant women?" : {
				"EN" : [
					"Yes"
				]
			},
			"Compatible with allergies?" : {
				"EN" : [
					"Yes"
				]
			},
			"q001" : {
				"EN" : [
					"3A",
					"3B",
					"3C",
					"4A",
					"4B",
					"4C"
				]
			},
			"q002" : {
				"EN" : [
					"Natural"
				]
			},
			"q003" : {
				"EN" : [
					"Curl definition",
					"Healthy hair",
				]
			},
			"q004" : {
				"EN" : [
					
                    "Dryness",
					"Frizz"
				]
			},
			"q005" : {
				"EN" : [
					"Oily",
					"Flaky",
					"Sensitive",
					"Dandruff",
					"Dermatitis",
					"Psoriasis"
				]
			}
		}
	}
}

"""

#Definition du questionnaire dans la variable questionnaire

questionnaire = """

{
	"questions" : [
		{
			"label" : "q001",
			"question" : {
				"EN" : "Texture(s) concerned"
			},
			"answers" : {
				"EN" : ["1A", "1B", "1C", "2A", "2B", "2C", "3A", "3B", "3C", "4A", "4B", "4C"]
			}
		},
		{
			"label" : "q002",
			"question" : {
				"EN" : "Condition(s)"
			},
			"answers" : {
				"EN": [
					"Natural", 
					"Straightened/chemically treated", 
					"In transition", 
					"Locs", 
					"Braids"]
			}
		},
		{
			"label" : "q003",
			"question" : {
				"EN" : "Desired objective"
			},
			"answers" : {
				"EN": [
					"Curl definition",
					"Length retention",
					"Moisture retention",
					"Shine enhancement",
					"Healthy heat styling",
					"Colour-treated hair care",
					"Manageability",
					"Stronger hair",
					"Volume enhancement",
					"Healthy hair",
					"Shrinkage",
					"None"
				  ]
			}
		},
		{
			"label" : "q004",
			"question" : {
				"EN" : "Problem encountered"
			},
			"answers" : {
				"EN": [
					"Product build-up",
					"Dryness",
					"Greasy hair",
					"Breakage",
					"Frizz",
					"Hair loss",
					"Dull hair",
					"Porous hair",
					"Heat damage",
					"Physical damage(pulling)",
					"Hair transition",
					"Colour change",
					"Manageability",
					"Thinning hair",
					"Weak edges",
					"None"
				  ]
			},
			"max_selections" : 3,
			"importance" : 4,
			"Tag" : "Hair Challenges"
		},
		{
			"label" : "q005",
			"question" : {
				"EN" : "Suitable scalp"
			},
			"answers" : {
				"EN": [
					"Dry", 
					"Oily", 
					"Flaky", 
					"Sensitive", 
					"Dandruff", 
					"Dermatitis", 
					"Alopecia", 
					"Psoriasis", 
					"None"
				]
			}
		}
	],
	"contraindications" : [
		{
			"contraindication" : "Ages involved",
			"answers" : {
				"EN" : [
					"0-1 year",
					"2-5 years",
					"6-12 years",
					"13-17 years",
					"18-24 years",
					"25-44 years",
					"45-64 years",
					"65 years and over"
                ]
			}
		},
		{
			"contraindication" : "Suitable for pregnant women?",
			"answers" : {
				"EN" : [
					"Yes",
					"No"
				]
			}
		},
		{
			"contraindication" : "Compatible with allergies?",
			"answers" : {
				"EN" : [
					"Yes",
					"No"
				]
		}
        }   
	]
}

"""

hair_type_dict_en = """

{
        '1A': 'straight and fine, known for its sleekness and smooth texture but may lack volume and get oily quickly',
        '1B': 'straight with some body, which holds styles better than finer hair and adds a bit more volume',
        '1C': 'straight with texture and body, making it versatile but prone to frizz in humid conditions',
        '2A': 'soft, loose waves that give your hair a gentle texture without too much frizz',
        '2B': 'wavy with more defined curls, giving your hair great texture and body but prone to frizz',
        '2C': 'wavy with thick, textured waves that bring volume and require moisture to maintain definition',
        '3A': 'curly with loose, well-defined curls that offer bounce and texture, requiring hydration for best results',
        '3B': 'curly with tighter ringlets that provide volume and definition but often need moisture to reduce frizz',
        '3C': 'curly with tight, springy curls that offer great texture but can shrink when dry, requiring intense moisture',
        '4A': 'coily with tight, well-defined curls that need deep hydration to avoid dryness and maintain strength',
        '4B': 'coily with less defined curls, offering volume and versatility but requiring moisture for definition',
        '4C': 'coily with very tight, zigzag curls, which thrive on intense moisture and need careful styling'
}
    
"""

#Declaration de la variable qui contient les informations sur le produit

product_information = f""" 


Recherche


Mon panier
0
Accueil
 
Boutique
 
Points de vente
 
Mon histoire
 
Cheb'Mag
 
FAQ
Accueil | huile | Bain d'huile - ChebElixir - 200 ml
Bain d'huile - ChebElixir - 200 ml
Bain d'huile - ChebElixir - 200 ml
Bain d'huile - ChebElixir - 200 ml

Bain d'huile - ChebElixir - 200 ml
 68 avis  Aucune question
FC 115,132.41 CDF
Taxes incluses. Frais d'expédition calculés lors du paiement.
Quantité
- 
1
 +

Plus de moyens de paiement
Votre allié pour gagner en volume 100% naturel. Votre couronne l'adore !
PAIEMENT SÉCURISÉ / LIVRAISON RAPIDE
Description du produit
Prenez soin de vos cheveux grâce aux propriétés nourrissantes de l’huile ChebElixir de ChebHair. Formulée à 100% de produits naturels, elle sera le nouvel allié quotidien de vos cheveux. Le chebe et la menthe poivrée favorisent la croissance de vos cheveux et les renforce grâce à leurs vertus ultra-hydratantes et fortifiantes. L’huile d’olive et le karité apportent une nutrition et une brillance intenses à vos cheveux. Fini les cheveux secs, ternes et cassants, l’huile ChebElixir apportera force, douceur et brillance à vos cheveux ! Elle s’adapte à tous types de cheveux, ondulés à crépus et même défrisés.



Détails du produit
SKU: CHEB-OIL200-CHEB
Poids: 200 g
Type de produit: huile
Marque: chebhairbycjv
Ingrédients
BUTYROSPERMUM PARKII OIL(karité/shea),RICINUS COMMUNIS SEED OIL(ricin/castor), ALLIUM CEPA BULB OIL(oignon/onion), OLEA EUROPAEA FRUIT OIL(olive), CROTON GRATISSIMUS SEED EXTRACT(chebe), MENTHA PIPERITA LEAF EXTRACT(menthe poivrée), CAPRYLIC/CAPRIC TRIGLYCERIDE, PARFUM, TOCOPHEROL(vitamine E/vitamin E), INULA CRITHMOIDE FLOWER/LEAF EXTRACT, LINALOOL

Utilisation
Pour un résultat optimal, secouez bien la bouteille avant chaque utilisation.
Pour un usage quotidien, appliquez une petite quantité de produit sur vos cheveux après votre crème ou lait capillaire.
Pour un soin nourrissant plus intense, appliquez une quantité plus généreuse en bain d’huile.

TEXTURE :
Liquide, légèrement épaisse.
.
AUTRES :
200ml
Bouteille plastique PET recyclable
Produit 100% naturel

.



PRECAUTIONS D’UTILISATION :Eviter tout contact avec les yeux – Ne pas avaler - Ne pas laisser à la portée des enfants. Afin d’éviter toute allergie il est conseillé de faire un test au préalable sur une petite partie de vos cheveux.

Partager:   
Bain d'huile - ChebElixir -...
FC 115,132.41 CDF
LIVRAISON RAPIDE

PAIEMENT SÉCURISÉ

100% NATUREL

FABRIQUÉ EN FRANCE

VOTRE ALLIÉ QUOTIDIEN
Le Chebelixir incarne l'excellence des soins capillaires naturels, spécialement conçu pour sublimer les cheveux texturés et crépus au quotidien. Cette formule unique, composée d'ingrédients naturels soigneusement sélectionnés, offre une expérience de soin inégalée.

Avantages pour votre routine capillaire
Pureté Naturelle : Le Chebelixir est formulé à partir d'ingrédients 100% naturels, offrant ainsi une approche authentique et respectueuse de la santé de vos cheveux.

Allié Quotidien : Grâce à sa formule légère et polyvalente, ce produit peut être utilisé quotidiennement pour nourrir, hydrater et revitaliser vos cheveux sans les alourdir.


Synergie d'Ingrédients : La combinaison de la menthe, de l'olive, du karité et du Chebe offre une synergie unique pour des cheveux visiblement plus sains et plus forts.

Le Chebelixir est bien plus qu'un simple produit capillaire, c'est un véritable élixir de beauté naturel pour sublimer vos boucles, favoriser leur croissance et lesmaintenir en pleine santé.



"""

style = "You are a cosmetics product data extractor."

prompt = f"""
  
Your task is to extract and return **only** the following product information from the provided product description text below.  
Use only the information that is **explicitely mentioned**.  
Do **not guess** or infer any data.  
Use only the exact values found in the allowed choices from the questionnaire or hair type dictionary.

Only include these fields in the output:

- Brand  
- Product name  
- Marketing Description  
- Key ingredients (can be listed or include in desciption)  
- Price (euros; CDF or another currency)  
- Quantity (ml)  
- Category  
- Ages involved (must match one or more age ranges in the questionnaire)  
- Suitable for pregnant women? ("Yes" or "No")  
- Compatible with allergies? ("Yes" or "No")  
- q001 – Hair texture(s) (must match keys from hair_type_dict_en)  
- q002 – Hair condition(s) (must match questionnaire)  
- q003 – Desired objective(s) (must match questionnaire)  
- q004 – Hair problem(s) (must match questionnaire)  
- q005 – Suitable scalp type(s) (must match questionnaire)

Return your result in this exact JSON format:

{{
  "Product Info": {{
    "Product Sheet": {{
      "Brand": "...",
      "Product name": "...",
      "Marketing Description": {{
        "EN": "..."
      }},
      "Key ingredients": {{
        "EN": [
          "..."
        ]
      }},
      "Price (euros)": "...",
      "Quantity (ml)": "...",
      "Category": {{
        "EN": "..."
      }},
      "Ages involved": {{
        "EN": [
          "..."
        ]
      }},
      "Suitable for pregnant women?": {{
        "EN": ["Yes" or "No"]
      }},
      "Compatible with allergies?": {{
        "EN": ["Yes" or "No"]
      }},
      "q001": {{
        "EN": [
          "..."
        ]
      }},
      "q002": {{
        "EN": [
          "..."
        ]
      }},
      "q003": {{
        "EN": [
          "..."
        ]
      }},
      "q004": {{
        "EN": [
          "..."
        ]
      }},
      "q005": {{
        "EN": [
          "..."
        ]
      }}
    }}
  }}
}}

Use only values that appear in the following references:
- Hair types (q001): {hair_type_dict_en}
- Questionnaire options: {questionnaire}

Source product description:

{product_information}

Return only the JSON output. Do not include comments, explanations, or introductory text.

"""

In [8]:
# Execute chat completion
start_time = time.time()
completion_model = "gpt-4o-mini" # Make sure it is deployed on Azure AI Studio
response = completeChat(prompt, style, client, completion_model)
elapsed_time = time.time() - start_time

# Print the response
print(response)
print(f"Time taken: {elapsed_time:.2f} seconds")

{
  "Product Info": {
    "Product Sheet": {
      "Brand": "chebhairbycjv",
      "Product name": "Bain d'huile - ChebElixir",
      "Marketing Description": {
        "EN": "Prenez soin de vos cheveux grâce aux propriétés nourrissantes de l’huile ChebElixir de ChebHair. Formulée à 100% de produits naturels, elle sera le nouvel allié quotidien de vos cheveux. Le chebe et la menthe poivrée favorisent la croissance de vos cheveux et les renforce grâce à leurs vertus ultra-hydratantes et fortifiantes. L’huile d’olive et le karité apportent une nutrition et une brillance intenses à vos cheveux. Fini les cheveux secs, ternes et cassants, l’huile ChebElixir apportera force, douceur et brillance à vos cheveux ! Elle s’adapte à tous types de cheveux, ondulés à crépus et même défrisés."
      },
      "Key ingredients": {
        "EN": [
          "BUTYROSPERMUM PARKII OIL",
          "RICINUS COMMUNIS SEED OIL",
          "ALLIUM CEPA BULB OIL",
          "OLEA EUROPAEA FRUIT OIL",
          

In [9]:
start_time = time.time()
embedding_model = "text-embedding-ada-002" # Make sure it is deployed on Azure AI Studio
embedding = embedText(response, client, embedding_model)
elapsed_time = time.time() - start_time

# Print the embeddings
print(embedding)
print(embedding.shape)
print(np.linalg.norm(embedding))
print(f"Time taken: {elapsed_time:.2f} seconds")

[-0.01302565  0.02317577 -0.00910806 ... -0.00409236 -0.01754341
 -0.02572154]
(1536,)
1.0
Time taken: 0.44 seconds


In [10]:
start_time = time.time()
first_keyword = "Bayesian Filtering"
second_keyword = "State Estimation"
query = f"Does the response talk about {first_keyword} and {second_keyword}?"
query_embedding = embedText(query, client, embedding_model)
similarity = np.dot(embedding, query_embedding) # Compute cosine similarity via inner product
elapsed_time = time.time() - start_time

# Print the similarity result
print(similarity)
print(f"Time taken: {elapsed_time:.2f} seconds")

0.6733676696335011
Time taken: 0.25 seconds
